In [1]:
import pickle
import numpy as np

In [2]:
import transformers as ppb
import torch

In [59]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
questions = pickle.load(open("questions", "rb"))
questions = list(questions)

In [4]:
query = 'cadastro no identity com endereço de email errado'

In [5]:
query_vec = bc.encode([query])[0]
# compute normalized dot product as score
score = np.sum(query_vec * doc_vecs, axis=1) / np.linalg.norm(doc_vecs, axis=1)
topk_idx = np.argsort(score)[::-1]

NameError: name 'bc' is not defined

In [63]:
score

array([10.100594, 10.369012, 10.139858, 11.049597, 10.474791],
      dtype=float32)

In [11]:
topk_idx

array([4, 2, 1, 0, 3])

In [12]:
questions[4]

'Um dos nossos usuários está cadastrado no fluig Identity e no fluig com o endereço de e-mail incorreto.Como faço para corrigir o endereço de e-mail do usuário nos dois sistemas?'

#### Importing the pre trained model

In [98]:
import warnings
warnings.filterwarnings('ignore')

model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-multilingual-cased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights,
                                    output_attentions = False, # Should the model returns attentions weights.
                                    output_hidden_states = True,  # Should the model returns all hidden-states.
                                   )

In [89]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_convert_head_mask_to_5d',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_generate_beam_search',
 '_generate_no_beam_search',
 '_get_name',
 '_get_resized_embeddings',
 '_hook_rss_memory_post_forward',
 '_hook_rss_memory_pre_forward',
 '_init_weights',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_prune_heads',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_reorder_cache',
 '_replicate_for_data_parallel',
 '_resize_token_embeddings',
 '_save_to_stat

#### Tokenization

In [99]:
tokenized = [tokenizer.encode(x, add_special_tokens=True) for x in questions[:5]]

In [100]:
# Print the original sentence.
print(' Original: ', questions[1])

# Print the sentence mapped to token ids.
print('Token IDs: ', ' '.join(tokenizer.convert_ids_to_tokens(tokenized[1])))

# Print the sentence split into tokens.
print('Tokenized: ', tokenized[1])

 Original:  Ao tentar alterar as informações de um usuário e bloquear o mesmo, uma mensagem é apresentada: result returns more than one elements. Por que essa mensagem aparece?
Token IDs:  [CLS] Ao tentar alter ##ar as informações de um usu ##ário e bloque ##ar o mesmo , uma mens ##agem é apresenta ##da : result returns more than one elements . Por que essa mens ##agem aparece ? [SEP]
Tokenized:  [101, 17607, 110694, 42141, 10354, 10146, 68263, 10104, 10293, 44224, 15211, 173, 99406, 10354, 183, 13544, 117, 10437, 12380, 29111, 263, 49320, 10229, 131, 14493, 38302, 10798, 11084, 10464, 17464, 119, 12399, 10121, 21553, 12380, 29111, 20962, 136, 102]


#### Padding

Pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [101]:
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized])

In [102]:
np.array(padded).shape

(5, 243)

In [103]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5, 243)

In [104]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

In [105]:
input_ids.shape

torch.Size([5, 243])

In [106]:
attention_mask.shape

torch.Size([5, 243])

In [107]:
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [110]:
len(last_hidden_states)

3

In [115]:
last_hidden_states[2][11].shape

torch.Size([5, 243, 768])

In [109]:
last_hidden_states[0].shape

torch.Size([5, 243, 768])

In [133]:
features = last_hidden_states[2][11][:,0,:].numpy()

In [134]:
len(features)

1

In [135]:
features[1]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [120]:
query = 'cadastro no identity com endereço de email errado'

In [121]:
max_len

243

In [123]:
tokenized_2 = [tokenizer.encode(query, add_special_tokens=True)]
padded_2 = np.array([i + [0]*(max_len-len(i)) for i in tokenized_2])
attention_mask_2 = np.where(padded_2 != 0, 1, 0)
input_ids_2 = torch.tensor(padded_2)  
attention_mask = torch.tensor(attention_mask)
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
query_vec = last_hidden_states[2][11][:,0,:].numpy()

In [124]:
query_vec[0]

array([ 3.36452067e-01, -1.13040343e-01,  7.46794403e-01, -4.00634319e-01,
       -8.88879418e-01,  1.93996459e-01, -2.48185202e-01,  4.87650722e-01,
       -5.16417205e-01,  4.93893713e-01,  8.33756804e-01, -7.26752341e-01,
       -1.94328487e-01, -3.33921403e-01, -3.46664727e-01, -2.56827436e-02,
       -2.40982771e-02,  1.15238571e+00,  1.49007857e-01, -9.33329985e-02,
        3.67020890e-02, -5.08620381e-01,  1.65550768e-01,  1.14915037e+00,
       -6.42913952e-02, -3.50975066e-01, -9.33264554e-01, -4.78302777e-01,
       -6.13736510e-01,  6.74391150e-01, -7.15082660e-02, -1.10006109e-01,
        1.03673875e+00,  4.41042960e-01, -3.78962398e-01,  4.24302340e-01,
       -1.14737727e-01, -9.32525635e-01, -7.69209146e-01, -8.18741173e-02,
       -4.67470214e-02,  1.07902920e+00, -4.71655011e-01,  2.05084682e-01,
        1.18104346e-01,  3.31115425e-01,  4.58963990e-01, -3.77871245e-01,
        2.34331220e-01, -5.74949265e-01,  9.45641220e-01, -3.23008716e-01,
        9.51558352e-01, -

In [125]:
# compute normalized dot product as score
score = np.sum(query_vec * features, axis=1) / np.linalg.norm(features, axis=1)
topk_idx = np.argsort(score)[::-1]

In [126]:
topk_idx

array([3, 2, 1, 4, 0])

In [128]:
questions[3]

'A mensagem FLUIG_REST_CLIENT_500 é mostrada no log do servidor constantemente antes da inicialização do fluig. Por que essas mensagens ocorrem?ERROR [com.fluig.wcm.core.WEBPageRouterBean] (default task-60) Error communicating with web-page API: com.fluig.core.exception.InternalServerErrorException: FLUIG_REST_CLIENT_500WARN [com.fluig.core.discovery.client.ServiceDiscoveryClient] (Timer-16) Discovery not available yet for i18n. Cause: FLUIG_REST_CLIENT_500. Help: Make sure that the server where fluig is installed has access to the address: http://[endereço fluig]:[porta]. Verifique se o servidor onde está instalado o fluig tem acesso ao endereço: http://[endereço fluig]:[porta]'

In [137]:
questions[4]

'Um dos nossos usuários está cadastrado no fluig Identity e no fluig com o endereço de e-mail incorreto.Como faço para corrigir o endereço de e-mail do usuário nos dois sistemas?'